## 制作用于DSAC*的数据集格式

In [19]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import numpy as np
import pycolmap
from tqdm import tqdm

from hloc.utils import viz_3d, parsers
from hloc.visualization import plot_images, read_image
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval, pairs_from_exhaustive, localize_sfm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))

1.13.0+cu117
11.7
8500
NVIDIA GeForce RTX 3050 Ti Laptop GPU


## 加载 SportsHall_Total 数据集

In [3]:
# 配置路径和参数
my_datasets = ['House11', 'SportsHall', 'SportsHall_Total']  # 11号楼、体育馆
sfm_dataset = Path('datasets/' + my_datasets[2])  # 用于三维重建的数据集

img_names = [r.relative_to(sfm_dataset).as_posix() for r in sfm_dataset.rglob("*/*")]  # 全部图像相对于数据集根目录的文件名

reference_imgs = sfm_dataset / 'reference/'  # 参考图像文件夹
reference_names = [r.relative_to(sfm_dataset).as_posix() for r in reference_imgs.rglob("*/*")]  # 相对于数据集根目录的文件名

query_imgs = sfm_dataset / 'query/'  # 查询图像文件夹
query_names = [q.relative_to(sfm_dataset).as_posix() for q in query_imgs.rglob("*/*")]  # 相对于数据集根目录的文件名

outputs = sfm_dataset / 'gt_hloc'  # 与官方不同，将outputs放置在数据集文件夹中

sfm_dir = outputs / 'Total_sfm_spsg'  # 对参考图像和查询图像一起重建得到的 SfM 模型
sfm_pairs = outputs / 'total_sfm_pairs.txt'  # 对参考图像和查询图像一起进行图像检索得到的配对
loc_pairs = outputs / 'loc_pairs.txt'  # 与查询图像配对的参考图像

retrieval = outputs / 'retrieval.h5'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

reference_intrinsics = outputs / 'reference_with_intrinsics.txt'  # 保存参考图像名称和估计的相机参数
query_intrinsics = outputs / 'query_with_intrinsics.txt'  # 保存查询图像名称和估计的相机参数
gt_reference_poses = outputs / 'gt_reference_poses.txt'  # 参考图像位姿真值（q4 t3）
gt_query_poses = outputs / 'gt_query_poses.txt'  # 查询图像位姿真值（q4 t3）

In [4]:
print(sfm_dir)
total_model = pycolmap.Reconstruction(sfm_dir)  # 加载三维重建模型
total_name_to_id = {img.name: i for i, img in total_model.images.items()}  # 所有图片的名称对应的id
print(total_model.summary())

datasets/SportsHall_Total/gt_hloc/Total_sfm_spsg
Reconstruction:
	num_reg_images = 1028
	num_cameras = 6
	num_points3D = 168024
	num_observations = 1181579
	mean_track_length = 7.0322
	mean_observations_per_image = 1149.4
	mean_reprojection_error = 1.29746


In [29]:
for camera_id, camera in total_model.cameras.items():
    print(camera_id, camera.summary())
    break

for image_id, image in total_model.images.items():
    print(image_id, image.summary())
    print(image.rotmat())
    print(image.projection_matrix())
    break

6 Camera:
	camera_id=6
	model = PINHOLE
	width = 1920
	height = 1080
	num_params = 4
	params_info = fx, fy, cx, cy
	params = 1348.938861, 1345.703944, 960.000000, 540.000000
5 Camera:
	camera_id=5
	model = PINHOLE
	width = 1920
	height = 1080
	num_params = 4
	params_info = fx, fy, cx, cy
	params = 1352.674412, 1346.690259, 960.000000, 540.000000
4 Camera:
	camera_id=4
	model = PINHOLE
	width = 1920
	height = 1080
	num_params = 4
	params_info = fx, fy, cx, cy
	params = 1350.234501, 1347.581837, 960.000000, 540.000000
3 Camera:
	camera_id=3
	model = PINHOLE
	width = 1920
	height = 1080
	num_params = 4
	params_info = fx, fy, cx, cy
	params = 1351.318503, 1348.960590, 960.000000, 540.000000
2 Camera:
	camera_id=2
	model = PINHOLE
	width = 1920
	height = 1080
	num_params = 4
	params_info = fx, fy, cx, cy
	params = 1333.949009, 1334.482231, 960.000000, 540.000000
1 Camera:
	camera_id=1
	model = PINHOLE
	width = 1920
	height = 1080
	num_params = 4
	params_info = fx, fy, cx, cy
	params = 1346.

## 保存到 DSAC 格式的数据集中

In [39]:
# 配置路径和参数
dsac_dataset = Path('datasets/' + my_datasets[2] + '_dsac')  # 用于三维重建的数据集

test_dir = dsac_dataset / 'test'
test_calibration = test_dir / 'calibration'
test_poses = test_dir / 'poses'

train_dir = dsac_dataset / 'training'
train_calibration = train_dir / 'calibration'
train_poses = train_dir / 'poses'

homo = np.array([0, 0, 0, 1])

In [40]:
# 保存测试集数据
for name in tqdm(query_names):
    image = total_model.find_image_with_name(name)
    camera = total_model.cameras[image.camera_id]
    save_name = name.split('/')[-1]
    save_name = save_name.split('.')[0] + '.txt'

    # 保存位姿
    T_w_c = image.projection_matrix()  # pycolmap位姿变换矩阵为从世界坐标系到相机坐标系
    T_w_c = np.vstack([T_w_c, homo])  # 写成齐次矩阵
    T_c_w = np.linalg.inv(T_w_c)  # DSAC*位姿变换矩阵为从相机坐标到场景坐标
    with open(test_poses / save_name, 'w', encoding='utf-8') as f:
        np.savetxt(f, T_c_w)

    # 保存相机参数
    focal = (camera.params[0] + camera.params[1]) / 2
    with open(test_calibration / save_name, 'w', encoding='utf-8') as f:
        np.savetxt(f, [focal])

100%|██████████| 120/120 [00:00<00:00, 5777.28it/s]


In [41]:
# 保存训练集数据
for name in tqdm(reference_names):
    image = total_model.find_image_with_name(name)
    camera = total_model.cameras[image.camera_id]
    save_name = name.split('/')[-1]
    save_name = save_name.split('.')[0] + '.txt'

    # 保存位姿
    T_w_c = image.projection_matrix()  # pycolmap位姿变换矩阵为从世界坐标系到相机坐标系
    T_w_c = np.vstack([T_w_c, homo])  # 写成齐次矩阵
    T_c_w = np.linalg.inv(T_w_c)  # DSAC*位姿变换矩阵为从相机坐标到场景坐标
    with open(train_poses / save_name, 'w', encoding='utf-8') as f:
        np.savetxt(f, T_c_w)

    # 保存相机参数
    focal = (camera.params[0] + camera.params[1]) / 2
    with open(train_calibration / save_name, 'w', encoding='utf-8') as f:
        np.savetxt(f, [focal])

100%|██████████| 908/908 [00:00<00:00, 6275.83it/s]
